## Spatial Data Science with CityJSON

The purpose of this Notebook is to ***work with*** the product of [osm_LoD1_3DCityModel](https://github.com/AdrianKriger/osm_LoD1_3DCityModel); a previously created CityJSON city model.

<div class="alert alert-block alert-warning"><b>This notebook will:</b>

> **1. allow the user to execute an application of Spatial Data Science**  
>
>> **a)  [population estimation](#Section1a)** _--with a previous census metric population growth rate and projected (future) population are also possible_  **and**  
>> **b)  a measure of [Building Volume per Capita](#Section1b)**
>
> **2. produce [an interactive visualization](#Section1b)** *-via [pydeck](https://deckgl.readthedocs.io/en/latest/)- which a user can navigate, query and share* **that**;
> > **a) [colour buildings by type](#Section2a)** *(to easily visualize building stock)* 
>
> **3. propose several [Geography and Sustainable Development Education *conversation starters*](#Section3) for Secondary and Tertiary level students**
</div>

In [1]:
#load the magic

%matplotlib inline
import os
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Polygon, shape, mapping
import json
import geojson

from cjio import cityjson

import matplotlib.pyplot as plt
import pydeck as pdk

In [2]:
#- works fine
import warnings
warnings.filterwarnings('ignore')

**The area under investigation is [Mamre, Cape Town. South Africa](https://en.wikipedia.org/wiki/Mamre,_South_Africa).**

In [3]:
#- change to harvest the appropriate CityJSON

#jparams = json.load(open('osm3DwStock_param.json'))
#jparams = json.load(open('osm3DsRiver_param.json'))
#jparams = json.load(open('osm3Dobs_param.json'))
jparams = json.load(open('osm3DuEstate_param5m.json'))
#jparams = json.load(open('osm3DwEstate_param.json'))      
#jparams = json.load(open('osm3Dcput_param.json'))       

In [4]:
cm = cityjson.load(path=jparams['cjsn_solid']) #-- citjsnClean_uEstate10m.json in the result folder

In [5]:
print(cm)

CityJSON version = 1.1
EPSG = 32734
bbox = [263780.58812656754, 6241099.216253888, 45.3, 6241099.216253888, 6241099.216253888, 167.2]
=== CityObjects ===
|-- TINRelief (1)
|-- Building (305)
materials = False
textures = False


In [6]:
df = cm.to_dataframe()
#- remove the first feature: the terrain
df = df[1:]            

gdf = gpd.GeoDataFrame(df, geometry=[shape(d) for d in df.pop("footprint")], crs=jparams['crs'])
#gdf.head(2)

In [7]:
gdf.head(2)

,osm_id,address,building,building:levels,plus_code,ground_height,building_height,roof_height,geometry
13705114,13705114,197 Nelson Mandela Boulevard Cape Town,warehouse,2,CPX8X9X2+XPV,70.0,6.9,76.9,"POLYGON ((264304.860 6241972.753, 264375.824 6..."
739615941,739615941,10 Rhodes Avenue University Estate Cape Town,house,2,CGXMXPX9+XQX,95.3,6.9,102.2,"POLYGON ((264275.999 6241813.835, 264278.104 6..."


<div class="alert alert-block alert-info"><b>Craveat</b> 
    
This community *(Woodstock, Salt River and Observatory)* is the second oldest community in South Africa. The buildings are old. Many have been repurposed. To account for refurbishment *--be as representative as possible--* and conform to the **[OpenStreetMap Guide](https://wiki.openstreetmap.org/wiki/Beginners%27_guide)** we typically tag these:  
`building=*` *~ the original purpose* `+` `building:use=*` *~ the current use*.

Furthermore; tagging in this community identifies **social housing, social facilities** (care home, shelter, etc.) **and informal housing** (backyard dwelling, shack, etc.) as `building / :use=residential`. **Student accomodation** includes the `residential=student` tag
</div>

In [8]:
#- to account for idiosyncratic mapping: replace building= (old function) if building:use= (new purpose) is present
gdf2 = gdf.copy()

#- 1
if 'building:use' in gdf.columns:
    df_res = gdf2[gdf2['building:use'] == 'residential']
    #df_res = gdf2[gdf2['building:use'] != None]
    df_res = df_res[~df_res['building:use'].isna()]
    gdf2.loc[df_res.index, 'building'] = df_res['building:use'] 

#- 2
#gdf2 = gdf.copy()
if 'residential' in gdf.columns:
    df_res = gdf2[gdf2['residential'] == 'student']
    #df_res = df2[df2['building:use'] != None]
    df_res = df_res[~df_res['residential'].isna()]
    gdf2.loc[df_res.index, 'building'] = df_res['residential'] 

## 1. Spatial Data Science

<div class="alert alert-block alert-warning"><b>We start with basic spatial analysis</b>  
    
     
- We'll [estimate the population](#Section1a), within our area of interest, and then  
- calculate the [Building Volume Per Capita (BVPC)](#Section1b).
</div>

While estimating population is well documented; recent investigations to **understand overcrowding** have led to newer measurements.  

The most noteable of these is **Building Volume Per Capita (BVPC)** [(Ghosh, T; et al. 2020)](https://www.researchgate.net/publication/343185735_Building_Volume_Per_Capita_BVPC_A_Spatially_Explicit_Measure_of_Inequality_Relevant_to_the_SDGs). BVPC is the cubic meters of building per person. **BVPC tells us how much space one person has per residential living unit** (a house / apartment / etc.). It is ***a proxy measure of economic inequality and a direct measure of housing inequality***.

BVPC builds on the work of [(Reddy, A and Leslie, T.F., 2013)](https://www.tandfonline.com/doi/abs/10.1080/02723638.2015.1060696?journalCode=rurb20) and attempts to integrate with several **[Sustainable Development Goals](https://sdgs.un.org/goals)** (most noteably: **[SDG 11: Developing sustainable cities and communities](https://sdgs.un.org/goals/goal11)**) and captures the average ***'living space'*** each person has in their home.

<div class="alert alert-block alert-info"><b>These analysis expect the user to have some basic knowledge about the environment under inquiry / investigation</b> </div>

In [9]:
#gdf.head(2)
len(gdf2)

305

In [10]:
#gdf.plot()
# have a look at the building type and amenities available
gdf2['building'].unique()

array(['warehouse', 'house', 'garage', 'yes', 'office'], dtype=object)

<a id='Section1a'></a>

<div class="alert alert-block alert-success"><b>1.  a) Estimate Population:</b> 
    
_(with population growth rate and population projection possible too)_ </div>

In [11]:
#- some data wrangling
with pd.option_context("future.no_silent_downcasting", True):
    if 'building:flats' in gdf2.columns: 
        gdf2['building:flats'] = pd.to_numeric(gdf2['building:flats'].fillna(0).infer_objects(copy=False))
    if 'building:units' in gdf2.columns:    
        gdf2['building:units'] = pd.to_numeric(gdf2['building:units'].fillna(0).infer_objects(copy=False), errors='coerce')
    if 'beds' in gdf2.columns:   
        gdf2['beds'] = pd.to_numeric(gdf2['beds'].fillna(0).infer_objects(copy=False))
    if 'rooms' in gdf2.columns:   
        gdf2['rooms'] = pd.to_numeric(gdf2['rooms'].fillna(0).infer_objects(copy=False))

gdf2["building:levels"] = pd.to_numeric(gdf2["building:levels"])

gdf2.head(2)

,osm_id,address,building,building:levels,plus_code,ground_height,building_height,roof_height,geometry
13705114,13705114,197 Nelson Mandela Boulevard Cape Town,warehouse,2.0,CPX8X9X2+XPV,70.0,6.9,76.9,"POLYGON ((264304.860 6241972.753, 264375.824 6..."
739615941,739615941,10 Rhodes Avenue University Estate Cape Town,house,2.0,CGXMXPX9+XQX,95.3,6.9,102.2,"POLYGON ((264275.999 6241813.835, 264278.104 6..."


In [12]:
#gdf2['building'].value_counts()

In [13]:
#--we only want building=house or =apartment or =residential
gdf_pop = gdf2[gdf2["building"].isin(['house', 'semidetached_house', 'terrace', 'apartments', 'residential', 'dormitory', 'cabin', 'student'])].copy()


In [14]:
#inf = gdf_pop[gdf_pop["building"].isin(['residential'])].copy()
len(gdf_pop)

295

In [15]:
#gdf_pop.plot()

**This area is urban with single and 2-storey level housing units. To estimate population is thus pretty straight forward.**

<div class="alert alert-block alert-info"><b>We start with local knowledge.</b></div>

**On average there are roughly `4` people per `building:house` in this area.**  

**[social housing](https://en.wikipedia.org/wiki/Public_housing)** is tagged `building:residential` with the `3` people per building or `building:flats * 3` if the building is an *apartment-type* complex

<div class="alert alert-block alert-danger"><b>Your Participation! </b>
    

We will execute the calculation programmatically. **Fill in the relevant variables in the _`cell`_ below** </div>

In [16]:
#- average number of residents per formal house
f_house = 4
#- average number of residents per informal structure
inf_structure = 3

<div class="alert alert-block alert-warning"><b></b>  
    
**Furthermore:**    
    - An ***informal*** structure ([shack](https://en.wikipedia.org/wiki/Shack)) is tagged [building:cabin](https://wiki.openstreetmap.org/wiki/Tag:building%3Dcabin) and houses `3` people.  
    - If a `building:residential` is a `social_facility` (carehome, shelter, etc.) it harvests the `beds` *'key:value'* pair.  
    - `building:apartment` harvests the `building:flats` *'key:value'* pair *(the number of units)* to calculate `*3` people per apartment.  
    - ***Student accomodation***:  
>    - University owed: is tagged `building:dormitory` with `residential:university` and harvests the `beds` or `rooms` *'key:value'* pair.
>    - Private for-profit: is tagged `building:residential` or `:dormitory` with `residential:university` and then harvests the `building:flats` or `:rooms` *'key:value'* pair *(the number of units)* to calculate `*1` people per apartment; if `level: > 1` else `*3` people in a house share.  

**The tagging scheme and numbers is based on *how your community is mapped* and local knowledge**
</div>

In [17]:
c = gdf_pop.columns

def pop(row):
    #- formal house
    if row['building'] == 'house' or row['building'] == 'semidetached_house':
        return f_house
    if row['building'] == 'terrace':
        return row['building:units'] * f_house

    #- informal structure (shack)
    if row['building'] == 'cabin':
        return inf_structure
        
    #- in this case social housing
    if row['building'] == 'residential' and 'social_facility' in c and row['social_facility'] is np.nan:
        if row['building:levels'] > 1:
            if 'rooms' in row and row['rooms'] != 0:
                return row['rooms']
            if 'building:flats' in row and row['building:flats'] != 0:
                return row['building:flats'] * inf_structure
        else:
            return inf_structure
    #-- social facility [shelter / carehome]
    if row['building'] == 'residential' and row['social_facility'] is not np.nan:
        if row['building:units'] != 0:
            return row['building:units'] * inf_structure
        else: 
            return row['beds']
                
    #- formal apartment
    if row['building'] == 'apartments':
        return row['building:flats'] * 3
        
    #- private student residence 
    if row['building'] == 'student':
        if row['building:levels'] > 1:
            return row['building:flats']
        else:
            return 3
    # university owned student residence
    if row['building'] == 'dormitory' and row['residential'] == 'university':
        if row['building:levels'] > 1:
            if 'rooms' in row and row['rooms'] != 0:
                return row['rooms']
            if 'beds' in row and row['beds'] != 0:
                return row['beds']
        else:
            return 3

gdf_pop['pop'] = gdf_pop.apply(lambda x: pop(x), axis=1)

est_pop = int(gdf_pop['pop'].sum())
print('The estimated population is:', est_pop)

The estimated population is: 1180


**[Statistics South Africa (STATSA)](https://www.statssa.gov.za) does not typically release official statistics at a suburb level but [City of Cape Town](https://www.capetown.gov.za/Family%20and%20home/education-and-research-materials/data-statistics-and-research/cape-town-census) generously publishes suburb profiles as Open Data.  
These numbers are based on disaggregated [Statistics South Africa (STATSA)](https://www.statssa.gov.za) [Census 2011](https://www.statssa.gov.za/?page_id=3839).** 

**Woodstock 12 656** (6 577: salt river and 9 207: observatory). 

We can calculate the annual population growth rate using the formula for **[Annual population growth](https://databank.worldbank.org/metadataglossary/health-nutrition-and-population-statistics/series/SP.POP.GROW):**

$$r = \frac{\ln{[\frac{End Population}{Start Population}}]}{n} * 100 = \frac{\ln{[\frac{17 062}{9345}}]}{12} * 100   = 2.49\%$$

<div class="alert alert-block alert-danger"><b>Your Participation! </b>
    

It is possible to execute the calculation programmatically. **Fill in the relevant variables in the _`cell`_ below** </div>

In [18]:
#- previous population
start_population = 12656

#- period in years from the previous census
years = 12

In [19]:
#-execute
r = (np.log(est_pop/start_population)/years) * 100
print('population growth rate of approximately:', round(r, 2), '%')

population growth rate of approximately: -19.77 %


To conclude; we can project into the future with a very basic formula to estimate the population _x_-years from now:  

$$p  = P_o * (1 + r)^{t} = p = 17062 * (1 + 0.0248)^{10}  = 21 818$$

<div class="alert alert-block alert-danger"><b>Your Participation! </b>
    

It is possible to execute the calculation programmatically. **Fill in the variables in the _`cell`_ below** </div>

In [20]:
#- period in years from now
years = 10

In [21]:
p = est_pop * (1 + (r/100))**years

print('estimated population', years ,'years from now:', int(p))

estimated population 10 years from now: 130


<a id='Section1b'></a>

<div class="alert alert-block alert-success"><b>1. b) Building Volume Per Capita (BVPC):</b>  
BVPC = total population of a community divided by sum of building volume</div>

In [22]:
#gdf_pop.head(3)

In [23]:
gdf_pop['area'] = gdf_pop['geometry'].area#\.map(lambda p: p.area)
gdf_pop['volume'] = gdf_pop['area'] * gdf_pop['building_height']

#- remove the volume of the ground floor (unoccupied) when building:levels > 7 [this is an arbitrary number based on local knowledge]
#- typically the space is reserved for some other function: retail, etc. 
gdf_pop['volume'] = [
    (row['volume'] - row['area'] * 2.8) if (
        'social_facility' in row and row['social_facility'] is np.nan and (row['building:levels'] > 7 and
        row['building'] in ['residential', 'apartments', 'student'])
    ) else row['volume']
    for _, row in gdf_pop.iterrows()
]

gdf_pop['bvpc'] =  gdf_pop['volume'] / gdf_pop['pop']

gdf_pop.tail(2)

,osm_id,address,building,building:levels,plus_code,ground_height,building_height,roof_height,geometry,pop,area,volume,bvpc
1025219390,1025219390,10 Kylemore Road University Estate Cape Town,house,2.0,CCX6X8XC+X7V,104.2,6.9,111.1,"POLYGON ((264231.363 6241791.926, 264231.955 6...",4,123.181468,849.952130,212.488032
1025219391,1025219391,2 Rhodes Avenue University Estate Cape Town,house,2.0,C9XXX2X6+XQW,101.2,6.9,108.1,"POLYGON ((264224.292 6241856.433, 264214.469 6...",4,105.446178,727.578631,181.894658


In [24]:
print(gdf_pop['bvpc'].describe())

count    295.000000
mean     213.642812
std      126.850648
min       34.491596
25%      129.096209
50%      179.227566
75%      265.138697
max      971.830674
Name: bvpc, dtype: float64


In [25]:
bvpc = round(gdf_pop['volume'].sum() / est_pop, 3)

print('Building Volume Per Capita (BVPC):', bvpc)

Building Volume Per Capita (BVPC): 213.643


In [26]:
#stu

<div class="alert alert-block alert-info"><b></b>

**This BVPC value is general.**  

We can seperate `building:house` from `building:cabin` and `building:residential` to undertand the differences between ***formal and informal*** housing in this area.
    
**We want to understand the living space *(the cubic-meter BVPC value)* each person has in thier home**
</div>

In [27]:
#gdf_pop[["osm_address", "osm_building:levels", "osm_beds", "bvpc"]]
#informal

In [28]:
formal = gdf_pop[gdf_pop["building"].isin(['house', 'semidetached_house', 'terrace', 'apartment'])].copy()
f_pop = int(formal['pop'].sum())
#f_area = formal['area'].mean()

informal = gdf_pop[gdf_pop["building"].isin(['residential', 'cabin'])].copy()
inf_pop = int(informal['pop'].sum())
#inf_area = formal['area'].mean()

#- student
stu = gdf_pop[gdf_pop["building"].isin(['student', 'dormitory'])].copy()
stu_pop = int(stu['pop'].sum())

#bvpc_formal = round(formal['volume'].sum() / est_pop, 3)
#bvpc_informal = round(informal['volume'].sum() / est_pop, 3)
#bvpc_stu = round(stu['volume'].sum() / est_pop, 3)

bvpc_formal = round(formal['volume'].sum() / formal['pop'].sum(), 3)
bvpc_informal = round(informal['volume'].sum() / informal['pop'].sum() if informal['pop'].sum() != 0 else 0, 3)
bvpc_stu = round(stu['volume'].sum() / stu['pop'].sum() if stu['pop'].sum() != 0 else 0, 3)

print('FORMAL: Population: ', f_pop, ' with Building Volume Per Capita (BVPC):', bvpc_formal)
print('')
print('STUDENT RESIDENCE: Population: ', stu_pop, ' with Building Volume Per Capita (BVPC):', bvpc_stu)
print('')
print('INFORMAL: Population: ', inf_pop, ' with Building Volume Per Capita (BVPC)', bvpc_informal)

FORMAL: Population:  1180  with Building Volume Per Capita (BVPC): 213.643

STUDENT RESIDENCE: Population:  0  with Building Volume Per Capita (BVPC): 0

INFORMAL: Population:  0  with Building Volume Per Capita (BVPC) 0


In [29]:
#informal

<div class="alert alert-block alert-danger"><b>Warning: </b>
    

These are LoD1 3D City Models and works well in these types of areas.  
LoD2 would offer a more representative BVpC [(Ghosh, T; et al. 2020)](https://www.researchgate.net/publication/343185735_Building_Volume_Per_Capita_BVPC_A_Spatially_Explicit_Measure_of_Inequality_Relevant_to_the_SDGs) value; when the complexity of the built environment increases.  

Think about a `house` with living space in the roof structure, so called *'attic living'*, or an `apartment` / `residential` building with different levels, loft apartments and/or units in the turrets of a `building`. 

***consider***: ***[geo3D](https://adriankriger.github.io/geo3D/)*** seperates [building:cabin](https://wiki.openstreetmap.org/wiki/Tag:building%3Dcabin) from `building:residential` to more precisely represent informal structures which typical do not have roof trussess but account for [social housing](https://en.wikipedia.org/wiki/Public_housing) that does</div></div>

<a id='Section2'></a>

## 2. Interactive Visualization

You might want to create and share an `html` visualization.

<div class="alert alert-block alert-warning"><b> </b>  
    
_In this example we identify building stock by **color** but you are limited only through your imagination and the data you have access too_
</div>

In [30]:
#- pydeck needs geographic coords
gdf = gdf2.to_crs(4326)

In [31]:
# -- get the location for pydeck
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    [xy] = gdf.dissolve().centroid

bbox = [gdf.total_bounds[0], gdf.total_bounds[1], 
        gdf.total_bounds[2], gdf.total_bounds[3]]

In [32]:
# have a look at the building type and amenities available
gdf['building'].unique()

array(['warehouse', 'house', 'garage', 'yes', 'office'], dtype=object)

<a id='Section2a'></a>
<div class="alert alert-block alert-success"><b>Building Stock:</b> To differentiate a school, housing, retail, healthcare and community focused facilities (library, municipal office, community centre) we color the buildings - we harvest the osm tags [amenity and building type] directly.</div>

In [33]:
#-- colour the building stock based on building:type

## while we can color with a built-in pydeck function
#color_lookup = pdk.data_utils.assign_random_colors(build_df['building'])
 # Assign a color
#build_df['color'] = build_df.apply(lambda row: color_lookup.get(row['building']), axis=1)

## we define specific colors
def color(bld):
    #- formal house
    if bld == 'house' or bld == 'semidetached_house':
        return [255, 255, 204]        #-grey
    #- informal structure / social housing
    if bld == 'residential' or bld == 'dormitory' or bld == 'cabin':
        return [119, 3, 252]          #-purple
    if bld == 'apartments':
        return [252, 194, 3]          #-orange 
    if bld == 'garage' or bld == 'parking':
        return [3, 132, 252]          #-blue        
    if bld == 'retail' or bld == 'supermarket':
        return [253, 141, 60]
    if bld == 'office' or bld == 'commercial':
        return [185, 206, 37]
    if bld == 'school' or bld == 'kindergarten' or bld == 'university' or bld == 'college':
        return [128, 0, 38]
    if bld == 'clinic' or bld == 'doctors' or bld == 'hospital':
        return [89, 182, 178]
    if bld == 'community_centre' or bld == 'service' or bld ==  'post_office' or bld == 'hall' \
    or bld ==  'townhall' or bld == 'police' or bld ==  'library':
        return [181, 182, 89]
    if bld == 'warehouse' or bld == 'industrial':
        return [193, 255, 193]
    if bld == 'restaurant' or bld == 'hotel':
        return [139, 117, 0]
    if bld == 'place_of_worship' or bld == 'church' or bld == 'mosque':
        return [225, 225, 51]
    else:
        return [255, 255, 204]

gdf["color"] = gdf['building'].apply(lambda x: color(x))

In [34]:
#- look
gdf.head(2)

,osm_id,address,building,building:levels,plus_code,ground_height,building_height,roof_height,geometry,color
13705114,13705114,197 Nelson Mandela Boulevard Cape Town,warehouse,2.0,CPX8X9X2+XPV,70.0,6.9,76.9,"POLYGON ((18.44997 -33.93634, 18.45072 -33.936...","[193, 255, 193]"
739615941,739615941,10 Rhodes Avenue University Estate Cape Town,house,2.0,CGXMXPX9+XQX,95.3,6.9,102.2,"POLYGON ((18.44961 -33.93777, 18.44964 -33.937...","[255, 255, 204]"


In [35]:
## ~ (x, y) - bl, tl, tr, br  ~~ or ~~ sw, nw, ne, se
#area = [[[18.4377, -33.9307], [18.4377, -33.9283], [18.4418, -33.9283], [18.4418, -33.9307]]]
area = [[[bbox[0], bbox[1]], [bbox[0], bbox[3]], 
         [bbox[2], bbox[3]], [bbox[2], bbox[1]]]]

## ~ (y, x)
view_state = pdk.ViewState(latitude=xy.y, longitude=xy.x, zoom=16.5, max_zoom=19, pitch=72, 
                                   bearing=80)

land = pdk.Layer(
    "PolygonLayer",
    area,
    stroked=False,
    # processes the data as a flat longitude-latitude pair
    get_polygon="-",
    get_fill_color=[0, 0, 0, 1],
    #material = True,
    #shadowEnabled = True
)
building_layer = pdk.Layer(
    "PolygonLayer",
    gdf,
    #id="geojson",
    opacity=0.3,
    stroked=False,
    get_polygon="geometry.coordinates",
    filled=True,
    extruded=True,
    wireframe=False,
    get_elevation="building_height",
    #get_fill_color="[255, 255, 255]", #255, 255, 255
    get_fill_color="color",
    get_line_color="color",#[255, 255, 255],
    #material = True, 
    #shadowEnabled = True, 
    auto_highlight=True,
    pickable=True,
)

tooltip = {"html": "<b>Levels:</b> {building:levels} <br/> <b>Address:</b> {address}\
<br/> <b>Plus Code:</b> {plus_code} <br/> <b>Building Type:</b> {building}"}

#change the tooltip to show bus routes and comment out the previous
#tooltip = {"html": "<b>Route:</b> {name} <br/>"}

r = pdk.Deck(layers=[land,  building_layer],#, greenspaces_layer, p_layer, water_layer, r_layer], #
             #views=[{"@@type": "MapView", "controller": True}],
             initial_view_state=view_state,
             map_style = 'dark_no_labels', #pdk.map_styles.LIGHT,
             tooltip=tooltip)
#save
r.to_html("./result/interactiveOnly.html")

**on a laptop without a mouse:**

- `trackpad left-click drag-left` and `-right`;
- `Ctrl left-click drag-up`, `-down`, `-left` and `-right` to rotate and so-on and
- `+` next to Backspace zoom-in and `-` next to `+` zoom-out.

**Now you do your community.** ~ If your area needs [OpenStreetMap](https://en.wikipedia.org/wiki/OpenStreetMap)  data and you want to contribute please follow the [Guide](https://wiki.openstreetmap.org/wiki/Beginners%27_guide).

<a id='Section3'></a>

<div class="alert alert-block alert-success"><b>3. Possible Secondary and Tertiary level conversations starters:</b></div>

| **Topic**                                | **Secondary Level Questions**                                                                                                                                                                                   | **Tertiary Level Questions**                                                                                                                                                                                                                   |
|------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Basic Understanding and Observations** | - What types of buildings are most common in the area (houses, apartments, retail, etc.)?<br>- Can you identify any patterns in the distribution of different types of buildings (e.g., are retail stores concentrated in certain areas)? | - How does the building stock composition (e.g., ratio of houses) correlate with the population? *demographics (e.g., age distribution, household size) for the area will strengthen the analysis!* <br>- Analyze the relationship between building density and population. What urban planning theories can explain this relationship? |
| **Spatial Relationships and Impacts**    | - How does the location of residential areas compare to the location of retail and commercial areas?<br>- What impact might the density and distribution of buildings have on local traffic and transportation?<br>- How might the population distribution affect the demand for local services such as schools, hospitals, and parks? | - Evaluate the accessibility of essential services (e.g., healthcare, education) in relation to the population and building types.<br>- Assess the potential social and economic impacts of a proposed new residential or commercial development in the area.                  |
| **Socioeconomic and Environmental Considerations** | - Are there any correlations between the types of housing available and the household size? *additional demographics (e.g., income level) for the area will strengthen the analysis!*<br>- How might the current building stock and population influence the local economy? *demographics (e.g., age distribution, household size) for the area will strengthen the analysis!*<br>- What are some potential environmental impacts of the current building distribution, such as green space availability or pollution levels? | - How does the current building stock support or hinder sustainable development goals (e.g., energy efficiency, reduced carbon footprint)?<br>- What strategies could be implemented to increase the resilience of the community to environmental or economic changes?                       |
| **Future Planning and Development**      | - Based on the current building stock and population metrics, what areas might benefit from additional housing or commercial development?<br>- How could urban planners use this information to improve the quality of life in the area?<br>- What changes would you recommend to better balance residential, commercial, and recreational spaces? | - How might different zoning regulations impact the distribution of residential, commercial, and industrial buildings in the future?<br>- Propose urban design solutions that could improve the sustainability and livability of the area, considering both current metrics and future projections. |
| **Quantitative and Qualitative Research** | |- Design a research study to investigate the impact of building type diversity on community wellbeing. What methodologies would you use?<br>- Analyze historical data to understand trends in building development and population growth. How have these trends shaped the current urban landscape?<br>- Conduct a SWOT analysis (Strengths, Weaknesses, Opportunities, Threats) of the area based on the building stock and population metrics. |